In [1]:
from __future__ import print_function
from collections import deque
from tensorflow.python.framework import ops
from rl_reinforce import REINFORCEothello
from othello_net import *
from othello_rules import *
from feature_extractor import *
import os
import tensorflow as tf
import numpy as np
import sys
import gym
from datetime import datetime
from example_states import *
from training_utils import *
np.set_printoptions(precision=2)


In [2]:
def sample_action(prediction):
    prediction = np.transpose(prediction[0])
    prediction = np.transpose(prediction[1])
    legal_moves = find_legal_moves(board, player)
    cleaned_predictions = zero_illegal_moves(prediction, legal_moves)
    p = cleaned_predictions.flatten()
    p = p / np.sum(p)
    sample_index = np.flatnonzero( np.random.multinomial(1,p,1) )[0]
    return moves[sample_index]

moves = ['0'] * 64
for i in range(8):
    for j in range(8):
        moves[i*8 + j] = str((i+1) * 10 + (j+1))

ops.reset_default_graph()

graph_1, img_data_1, train_step_1, optimizer_1, ground_truths_1, loss_1, pred_up_1, keep_prob_1, learn_rate_1, score_out_1 = create_othello_net()
sess_1 = tf.Session(graph=graph_1)
saver_1 = tf.train.Saver()
init_op_1 = tf.initialize_all_variables()
sess_1.run(init_op_1)
current_model = "models/tiny-selfplay.ckpt"
if os.path.isfile(current_model):
    saver_1.restore(sess_1, current_model)

#ops.reset_default_graph()
graph_2, img_data_2, train_step_2, optimizer_2, ground_truths_2, loss_2, pred_up_2, keep_prob_2, learn_rate_2, score_out_2 = create_othello_net()
sess_2 = tf.Session(graph=graph_2)
saver_2 = tf.train.Saver()
init_op_2 = tf.initialize_all_variables()
sess_2.run(init_op_2)
#current_model = "models/tiny-selfplay-2.ckpt"
current_model = current_model
if os.path.isfile(current_model):
    saver_2.restore(sess_2, current_model)
    
N = 100
graph_1 = -1
graph_2 = 1
graph_1_wins = 0
graph_2_wins = 0
for n in range(N):
    if n % 100 is 0:
        print(n)
    board = initialize_game()
    player = -1
    while True:
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            #print("game is over")
            #print(board)
            winner = get_winner(board, 1, -1)
            #print("winner is " + str(winner))
            if winner is graph_1:
                graph_1_wins += 1
            if winner is graph_2:
                graph_2_wins += 1
            break
        features = board_to_input(board, player)
        if player is graph_1:
            prediction = sess_1.run(pred_up_1, feed_dict={img_data_1:[features], keep_prob_1:1.0})
        else:
            prediction = sess_2.run(pred_up_2, feed_dict={img_data_2:[features], keep_prob_2:1.0})
        sampled_move = sample_action(prediction)
        
        
        
        board = make_move(board, sampled_move, player)

        if player is 1:
            player = -1
        else:
            player = 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            if player is 1:
                player = -1
            else:
                player = 1
print("graph 1 wins are " + str(graph_1_wins))
print("graph 2 wins are " + str(graph_2_wins))


0
graph 1 wins are 39
graph 2 wins are 56


In [3]:
#Games against itself:
#graph 1 wins are 39
#graph 2 wins are 56

